In [1]:
# Tested with Python 3.11, OpenCV 4.9, NumPy 1.26 and SciPy 1.12
# pip install opencv-python numpy scipy matplotlib ipykernel
# Windows: If "DLL load failed", get a redist from:
#   https://learn.microsoft.com/en-us/cpp/windows/latest-supported-vc-redist
#   Or possibly by: pip install msvc-runtime

# Local
import resources.image_interpolator as tools

# Math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mlab

# Data
import cv2 as cv
import re

In [2]:
# Get image from file
file = 'Source/Vary Clouds (South Horizon)/#9 2024-04-08 13.05.13.png'
taken = re.findall(r'\d{4}-\d{2}-\d{2} \d{2}\.\d{2}\.\d{2}', file)[0]
Img1, Img2 = np.hsplit(cv.imread(file)[:,:,::-1], 2)

# Image Settings
imgH, imgW = Img1.shape[:2]
downscale = imgH/20

# Display Settings
np.set_printoptions(3, linewidth=200)
mlab.rc('legend', fontsize=12)
mlab.rcParams['axes.formatter.use_mathtext'] = True
plt.rc('font', family='serif', serif='cmr10', size=plt.rcParams['legend.fontsize']+4)
plt.rc('axes', unicode_minus=False)
legendBBox = dict(boxstyle='round,pad=.4,rounding_size=.2', fc='w', ec=(.8,.8,.8), alpha=.8)
cBarPad = .01
scatMS = 4

In [3]:
# Draw box(es) with text(s) on an OpenCV image
def cvTextBox(img, text, loc=(20, 20), margin=(3, 6), alpha=.4, font=cv.FONT_HERSHEY_DUPLEX, size=1, fc=(0,0,0), bc=tools.maxC, thickness=2, lineType=cv.LINE_AA):
    if isinstance(text, str):
        text = [text]
    if img.shape[-1] > tools.ch:
        fc = (*fc, tools.maxC)
    size *= img.shape[1] / 2000
    margin = (size*margin[0], size*margin[1])
    thickness = max(int(size*thickness), 1)

    # Calculate positions and add background boxes
    textLoc = [(0,0)]*len(text)
    l, t = loc # Left, Top
    for i, label in enumerate(text):
        w, h = cv.getTextSize(label, font, size, thickness)[0] # Width, Height
        r, b = int(l+w+2*margin[0]), int(t+h+2*margin[1]+2) # Right, Bottom
        sub = img[t:b,l:r,:tools.ch].astype(np.float32)
        img[t:b,l:r,:tools.ch] = (sub*(1-alpha)+np.ones_like(sub)*bc*alpha).astype(img.dtype) # cv.AddWeighted inconsistently throws "int() argument must be a string" (memory bug in CV's Python-C++ interface?)
        textLoc[i] = [int(l+margin[0]), int(t+h+margin[1]-1)]
        t = b

    # Loop again to gaurantee that labels are in front
    for i, label in enumerate(text):
        cv.putText(img, label, textLoc[i], font, size, fc, thickness, lineType)

# Draw an (anchored) box with text on a plt axes
def pltTextBox(ax, text, loc, bbox=legendBBox, size=plt.rcParams['legend.fontsize']):
    parambox = mlab.offsetbox.AnchoredText(text, loc, prop=dict(bbox=bbox, size=size))
    parambox.patch.set_alpha(0)
    ax.add_artist(parambox)

# Calculate the distances and angles between each pair in two lists of points
def points2DistAng(p1, p2):
    vectors = p1-p2
    distances = np.linalg.norm(vectors, axis=0)
    angles = np.degrees(np.arctan(vectors[1]/vectors[0]))
    return distances, angles

# Filter out matches lower than median similarity
def MedSim(matches, inStr='m'):
    filtered = sorted(matches, key=lambda m: m.distance)[:len(matches)//2]
    return filtered, 'MedSim(%s)' % inStr

# Filter with RANSAC (t: Maximum allowed reprojection error to treat a point pair as an inlier)
def RANSAC(matches, inStr='m', t=30):
    p1, p2 = tools.getMatchPoints(matches, kp1, kp2)
    _, labels = cv.findHomography(p1.T, p2.T, cv.RANSAC, t)
    filtered = [matches[i] for i,l in enumerate(labels) if l == 1]
    return filtered, 'RANSAC(%s, t=%d)' % (inStr, t)

In [ ]:
# Find keypoints in images
featureFinder = cv.SIFT_create()
kp1, des1 = featureFinder.detectAndCompute(cv.cvtColor(Img1[:,:,::-1], cv.COLOR_BGR2GRAY), None)
kp2, des2 = featureFinder.detectAndCompute(cv.cvtColor(Img2[:,:,::-1], cv.COLOR_BGR2GRAY), None)

# Plot images with keypoints
img = np.concatenate((cv.drawKeypoints(Img1[:,:,::-1], kp1, None), cv.drawKeypoints(Img2[:,:,::-1], kp2, None)), axis=1)
cvTextBox(img, ['Image: '+taken, 'Method: '+featureFinder.getDefaultName(), '%i features'%len(kp1)])
cvTextBox(img, '%i features'%len(kp2), (imgW+20, 20))
fig = plt.figure(figsize=[img.shape[dim]/downscale for dim in (0,1)]); plt.axis('off')
plt.imshow(img[:,:,::-1]); tools.save('1 Detect', img)

In [ ]:
# Match keypoints & evaluate filtering
matcher = cv.BFMatcher(crossCheck=True)
matches = matcher.match(des1, des2)
mName = matcher.__class__.__name__

# Dummy function, returns the unfiltered matches
def unfiltered(matches):
    return matches, 'Unfiltered'

# Declarations used for plotting
lim = [f(array) for array in points2DistAng(*tools.getMatchPoints(matches, kp1, kp2)) for f in (min, max)]
textLoc = [a+c*(b-a) for a,b,c in ((*lim[:2], -.02), (lim[3], lim[2], .08))]
cMap = plt.cm.viridis; norm = mlab.colors.LogNorm(1, len(matches))

filters = [[unfiltered, MedSim], [RANSAC, lambda m: RANSAC(*MedSim(m))]]
fig, axes = plt.subplots(len(filters), len(filters[0]), sharex=True, sharey=True, gridspec_kw=dict(wspace=0, hspace=0), subplot_kw=dict(box_aspect=1), figsize=(5.36, 5.25))

# Plot distance-angle distributions at each filter stage
for i,row in enumerate(axes):
    for j,ax in enumerate(row):
        m, filterMethod = filters[i][j](matches)
        scale = ax.hexbin(*points2DistAng(*tools.getMatchPoints(m, kp1, kp2)), gridsize=40, extent=lim, cmap=cMap, norm=norm, lw=.2)
        pltTextBox(ax, '%s\n%d st'%(filterMethod, len(m)), 'upper right', size=10)
fig.colorbar(scale, ax=axes, aspect=71, fraction=.014, pad=cBarPad, format='{x:.0f}', label='Number of matches')
axes[0,0].set_ylabel('Angle relative to horizontal (degrees)'); axes[0,0].yaxis.set_label_coords(-.25, 0);
axes[1,0].set_xlabel('Distance between matched points (pixels)'); axes[1,0].xaxis.set_label_coords(1.01, -.2);
tools.save('2 Match', fig)

In [ ]:
# Plot images with filtered matches
m, mFilter = RANSAC(*MedSim(matches))
img = cv.drawMatches(Img1[:,:,::-1], kp1, Img2[:,:,::-1], kp2, m, None, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
legend = ['Image: '+taken, mName+': %i matches (not shown)'%len(matches), mFilter+': %i matches'%len(m)]
cvTextBox(img, legend, (20, imgH-220))

plt.figure(figsize=[img.shape[dim]/downscale for dim in (0,1)]); plt.axis('off')
plt.imshow(img[:,:,::-1]); tools.save('2 Match', img)

In [ ]:
# Calculate and display alignment from keypoints
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, constrained_layout=True, figsize=(12, 4.3))
cMap = plt.cm.rainbow; fig.gca().invert_yaxis()
filterMatches, _ = RANSAC(*MedSim(matches), 30)
order = 2; minAlgo = 'Powell'; tol = 1e-18
c1 = c2 = np.zeros((tools.polyTerms2D(order), 2))

p1, p2 = tools.getMatchPoints(filterMatches, kp1, kp2)
p1G, p2G, translation, rotation, center, angle = tools.globalAlignment(p1, p2)
errorG = np.linalg.norm(p1G-p2G, axis=0)
p1L, p2L, c1, c2 = tools.localAlignment(p1G, p2G, c1, c2, order, minAlgo, tol)
errorL = np.linalg.norm(p1L-p2L, axis=0)
p1G[0] += imgW/2; p2G[0] += imgW/2; p1L[0] += imgW/2; p2L[0] += imgW/2

# Plot the coordinates of filtered matches
ax1.scatter(*p1, scatMS), ax1.scatter(p2[0]+imgW, p2[1], scatMS)
ax1.set_ylabel('Y-coordinates of matches'); ax1.set_xlabel('X-coordinates ('+str(len(filterMatches))+' matches)')
ax1.legend(['Left image', 'Right image'], loc='upper right', markerscale=3, labelspacing=.2, handlelength=1, handletextpad=.3)

# Add center and angle to first plot
arrowprops = dict(arrowstyle='->', lw=.7)
arrow = [[0],[-250]]; refLines = [[0,0,200],[-200,0,0]]; center2pW = center[1]+[[imgW], [0]]
arrow1 = tools.rotMat(angle[0]) @ arrow + center[0]; ref1 = np.append(center[0], refLines+center[0], axis=1)
arrow2 = tools.rotMat(angle[1]) @ arrow + center2pW; ref2 = np.append(center2pW, refLines+center2pW, axis=1)
ax1.plot(*ref1, 'k,--', *ref2, 'k,--', ms=15, lw=.7)
ax1.annotate('', arrow1[:,0], center[0][:,0], arrowprops=arrowprops)
ax1.annotate('', arrow2[:,0], center2pW[:,0], arrowprops=arrowprops)

# Plot affine tranformation
labelG = ['Rotation: %.3f rad'%((angle[0]-angle[1])/2), 'Translation: (%i, %i)'%tuple(translation[0].ravel()), 'Mean error: %.2f' % np.mean(errorG)]
for i, e in enumerate(errorG):
    if e > 6:
        ax2.plot(*zip(p1G.T[i], p2G.T[i]), 'k--', lw=.5)
scale = ax2.scatter(*np.append(p1G, p2G, axis=1), scatMS, np.tile(errorG, 2), cmap=cMap)
pltTextBox(ax2, '\n'.join(labelG), 'upper right')
ax2.set_xlabel('       X-coordinates after affine transformation'); ax2.set_aspect('equal')
fig.colorbar(scale, ax=ax2, pad=cBarPad)

# Plot warp connections and filter out graphical redundancy
delIdxs = []
for i, e in enumerate(errorL):
    if e > 6: # Approximately the distance of two radices
        ax3.plot(*zip(p1L.T[i], p2L.T[i]), 'k--', lw=.5)
    elif e < 3: # Approximately the distance where point center is outside match point's radius
        delIdxs.append(i)
errorLMean = np.mean(errorL)
errorLFilt = np.delete(np.tile(errorL, 2), np.array(delIdxs)+len(p1L[0]))
p2LFilt = np.delete(p2L, delIdxs, axis=1)

# Plot warping
scale = ax3.scatter(*np.append(p1L, p2LFilt, axis=1), scatMS, errorLFilt, cmap=cMap)
pltTextBox(ax3, 'Algorithm: %s\nMean error: %.2f'%(minAlgo, errorLMean), 'upper right')
ax3.set_xlabel('X-coordinates after warping'); ax3.set_aspect('equal')
cbar = fig.colorbar(scale, ax=ax3, pad=cBarPad, label='Distance between matches')

tools.save('3 Align', fig)

In [ ]:
# Apply affine transformation to the images
p0 = np.array([np.repeat(range(imgW), imgH), np.tile(range(imgH), imgW)])
p1, p2, _, _, _, _ = tools.globalAlignment(p0, p0, translation, rotation, center)
img, _, percent, _ = tools.overlayImages(p0, p1, p2, [Img1, Img2])

legend = ['Image: '+taken] + labelG[::-1] + ['Overlap: %i%%' % percent]
cvTextBox(img, legend, alpha=.5)

fig = plt.figure(figsize=[img.shape[dim]/downscale for dim in (1,0)]); plt.axis('off')
plt.imshow(img.astype(int)); tools.save('4 Affine', img)

In [ ]:
# Apply warping to the images
img, diff, overlap, _, _ = tools.mergeAndWarpImages(p0, p0, c1, c2, order, translation, rotation, center, [Img1, Img2])

legend = ['Image: '+taken, 'Algorithm: '+minAlgo, 'Mean match error: %.2f'%np.mean(errorL)]
cvTextBox(img, legend, alpha=.5); cvTextBox(diff, legend[:2])

fig = plt.figure(figsize=[img.shape[dim]/downscale for dim in (1,0)]); plt.axis('off')
plt.imshow(img.astype(int)); tools.save('5 Warp', img)

fig = plt.figure(figsize=[img.shape[dim]/downscale for dim in (1,0)]); plt.axis('off')
plt.imshow(diff.astype(int)); tools.save('6 Diff', diff)